## Descrição

Notebook onde se cria uma tabela onde é registado por dia, tipo e circuito todos os pontos que foram observados, recolhidos, a sua ultima observação, a sua última recolha e os deltas

In [2]:
import pandas as pd
from datetime import datetime, time
import numpy as np

Dependências : tabela_dados_nivel_recolha_ordenado_v2.csv


Tempo médio de processamento: 90 minutos

In [3]:
# Carregar os dados
df = pd.read_csv("../data/tabela_dados_nivel_recolha_ordenado_v2.csv", delimiter=',')
df['Data do circuito'] = pd.to_datetime(df['Data do circuito'])

# Filtrar até final de janeiro
#df = df[df['Data do circuito'].dt.date <= datetime(2024, 1, 31).date()]

In [5]:
# Preparar estruturas
df['Data'] = df['Data do circuito'].dt.date
tipos_recolha = {
    'Papel': 'Recolhido Papel',
    'Embalagens': 'Recolhido Embalagens',
    'Vidro': 'Recolhido Vidro'
}
mapa_pontos = df.groupby('Circuito')['local_id'].unique().apply(list).to_dict()
grupos = df.groupby(['Data', 'Circuito'])

In [7]:
a = 0
b = 0
c = 0
d = 0

In [11]:
# Função: obter nível no início do dia
def obter_nivel_inicio(data, ponto, tipo):
    inicio_dia = datetime.combine(data, time(0, 0))
    dados = df[(df['local_id'] == ponto) & (df['Data do circuito'] < inicio_dia)]
    dados = dados.sort_values(by='Data do circuito')
    if not dados.empty and pd.notna(dados[tipo]).any():
        return dados[tipo].dropna().iloc[-1]
    return pd.NA

In [13]:
registos = []
#Cria uma lista onde vão ser guardados dicionários com os dados finais, um por ponto, por tipo e por circuito por dia.

for (data, circuito), grupo in grupos:
    if pd.isna(circuito):
        continue
    #Percorre cada grupo no DataFrame agrupado por (Data, Circuito):
    #data → a data da execução do circuito
    #circuito → nome do circuito
    #grupo → subconjunto do DataFrame original apenas com aquela data e aquele circuito
    #Se o cirucito não existir nessa data, ignora

    
    for tipo, col_recolhido in tipos_recolha.items():
        if (grupo[col_recolhido] == 1).any():
            #Percorre os 3 tipos de resíduos e verifica se houve alguma recolha desse tipo nesse circuito específico 
            #(valores 1 na coluna Recolhido <Tipo>)
            #Se houve é porque o cirucito foi feito
            pontos = mapa_pontos.get(circuito, [])
            #Obtém os pontos associados a este circuito (do dicionário mapa_pontos) e itera sobre cada um deles.
            
            for ponto in pontos:
                nivel_inicio = obter_nivel_inicio(data, ponto, tipo)
                #Calcula o nível que o ponto tinha no início do dia
                fim_dia = datetime.combine(data, time(23, 59, 59))
                #Define o fim do dia da data em questão, para considerar todos os registos até esse dia.
                
                registos_dia = df[
                    (df['local_id'] == ponto) &
                    (df['Data do circuito'] <= fim_dia)
                ].sort_values(by='Data do circuito')
                #Seleciona todos os registos do ponto até ao final do dia (pode ser em qualquer circuito), ordenados pela data.

                # Apenas registos válidos para o tipo
                registos_tipo_validos = registos_dia[pd.notna(registos_dia[tipo])]

                # Verifica se houve recolha naquele circuito e data para o ponto
                recolhido_no_circuito = grupo[
                    (grupo['local_id'] == ponto) &
                    (grupo[col_recolhido] == 1)
                ]

                # NOVO: Verifica se o ponto apareceu em qualquer circuito nesse dia
                registos_no_dia = df[(df['Data'] == data) & (df['local_id'] == ponto)]
                #Agora em vez de estar a procurar se houve registos no grupo, procuro se houve registos no dia
                
                # --- CASO 1: O ponto não foi registado em nenhum circuito nesta data ---
                if registos_no_dia.empty:
                    nivel_fim = nivel_inicio
                    recolhido = 0
                    atualizado = 0
                    a += 1
                #Se não houve nenhum registo do ponto nesse tipo até ao final do dia, o nivel mantem-se.
                #E não houve nem recolha nem observação.
            
                
                # --- CASO 2: Houve recolha neste circuito ---
                elif not recolhido_no_circuito.empty:
                    reg_final = recolhido_no_circuito.sort_values('Data do circuito').iloc[-1]
                    #Os registos são ordenados por data (sort_values).
                    #iloc[-1] pega o último (ou seja, o mais recente até ao fim do dia).
                    #Esse será o momento onde ocorreu a recolha.
                    
                    # Verifica se o nível não é NaN
                    if pd.notna(reg_final[tipo]):
                        nivel_fim = reg_final[tipo] #Se for não for NaN, o nível é atribuído a nivel_fim
                    else:
                        nivel_fim = pd.NA #Se for NaN atribui-se NaN

                    recolhido = 1
                    atualizado = 1
                    b += 1

                # --- CASO 3: Houve observação mas não recolha ---
                else:
                    if not registos_tipo_validos.empty:
                        reg_final = registos_tipo_validos.iloc[-1]
                        nivel_fim = reg_final[tipo]
                        #Se houve observação, mas não houve recolha, coloca-se o nível que foi registado. 
                        #Se o nível for NaN, vai se colocar o ultimo valor não NaN
                    else:
                        nivel_fim = nivel_inicio
                        d +=1
                        #Se não houver valor diferente de NaN, atribui-se que o valor final é igual ao inicial
                    recolhido = 0
                    atualizado = 1
                    c += 1

                registos.append({
                    "Data": data,
                    "Circuito": circuito,
                    "Tipo": tipo,
                    "local_id": ponto,
                    "Nivel_inicio": nivel_inicio,
                    "Nivel_fim": nivel_fim,
                    "Recolhido": recolhido,
                    "Observacao": atualizado
                })

In [17]:
resultado_df = pd.DataFrame(registos)

In [19]:
print("Caso 1: ", a)
print("Caso 2: ", b)
print("Caso 3: ", c)
print("Especial: ", d)

Caso 1:  120322
Caso 2:  122346
Caso 3:  119696
Especial:  379


In [64]:
def calcular_tempo_desde_ultima_acao(resultado_df, df, coluna_flag_origem, coluna_destino):
    df = df.copy()
    df['DataHora'] = pd.to_datetime(df['Data do circuito'].astype(str) + ' ' + df['Hora'].astype(str))
    resultado_df[coluna_destino] = pd.NaT

    for idx, linha in resultado_df.iterrows():
        ponto = linha['local_id']
        tipo = linha['Tipo']
        circuito = linha['Circuito']
        data = linha['Data']
        data_inicio = datetime.combine(data, time(0, 0))

        # Condições base: ponto e registos antes do início do dia
        condicoes = (
            (df['local_id'] == ponto) &
            (df['DataHora'] < data_inicio)
        )

        # Se for recolhido, filtra também pelo mesmo circuito
        if coluna_flag_origem == 'Recolhido':
            condicoes &= (df['Circuito'] == circuito)

        df_filtrado = df[condicoes]

        # Escolher os registos com a ação
        if coluna_flag_origem == 'Recolhido':
            df_validos = df_filtrado[df_filtrado[tipos_recolha[tipo]] == 1]
        else:
            df_validos = df_filtrado[pd.notna(df_filtrado[tipo])]

        # Obter o último (mais recente)
        if not df_validos.empty:
            ultima_datahora = df_validos['DataHora'].max()
            resultado_df.at[idx, coluna_destino] = data_inicio - ultima_datahora
        else:
            resultado_df.at[idx, coluna_destino] = pd.NaT



Esta função calcula quanto tempo passou desde a última observação ou recolha, até ao início do dia daquela linha.

Cria-se uma cópia do df para não alterar o original.
Cria-se a coluna 'DataHora' onde se junta a data e a hora.
Inicializa-se a nova coluna de destino (ex: 'ultima_observacao') com valores vazios (NaT = Not a Time)

Percorre cada linha do resultados_df e extrai-se o ponto, tipo, circuito e data. Define-se o inicio do dia correpondente à linha atual 

As condições servem para filtrar apenas os registos antes do início do dia e para o ponto atual.

Se a coluna flag for Recolhido, então no filtro inclui-se também o circuito

Cria-se um dataframe com as condições

Se a coluna flag for Recolhido, filtra-se apenas as linhas onde houve recolha para aquele tipo, antes do início do dia. (df_validos)

Se for Observacao (else), vai buscar a ultimo registo para o tipo, que seja diferente de NaN (df_validos)

df_validos['DataHora'].max(): 

Esta linha obtém o timestamp mais recente (ou seja, o último) entre os registos válidos antes do início do dia.


data_inicio - ultima_datahora:

Calcula a diferença de tempo entre o início do dia atual (00:00h) e a última ação registada.



resultado_df.at[idx, coluna_destino]:

Atribui o tempo decorrido diretamente à linha correspondente no resultado_df, na nova coluna ('ultima_observacao' ou 'ultima_recolha').

É atribuido NaT quando não existiu nenhum registo válido antes do início do dia, ou seja, df_validos.empty é True

Exemplo:

Se uma observação foi feita no dia 02-03-2024 às 13:30:00, no dia 03-03-2024 vai aparecer na coluna ultima_observação que
a ultima observação foi feita à 0 days 10:30:00

In [25]:
# Aplicar para observacao e recolha
calcular_tempo_desde_ultima_acao(resultado_df, df, 'Recolhido', 'ultima_recolha')
calcular_tempo_desde_ultima_acao(resultado_df, df, 'Observacao', 'ultima_observacao')

C:\Users\simao\AppData\Local\Temp\ipykernel_15684\3112035281.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0 days 19:27:30' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  resultado_df.at[idx, coluna_destino] = data_inicio - ultima_datahora
C:\Users\simao\AppData\Local\Temp\ipykernel_15684\3112035281.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0 days 19:27:46' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  resultado_df.at[idx, coluna_destino] = data_inicio - ultima_datahora


In [28]:
resultado_df["deltas"] = resultado_df["Nivel_fim"] - resultado_df["Nivel_inicio"]

In [32]:
inconsistencias = resultado_df[resultado_df["Nivel_inicio"] > resultado_df["Nivel_fim"]]
print(inconsistencias)

              Data     Circuito        Tipo  local_id Nivel_inicio Nivel_fim  \
887     2024-01-03  Circ. Ilhas       Vidro      3844          2.0       1.0   
893     2024-01-03  Circ. Ilhas       Vidro      3681          2.0       1.0   
925     2024-01-03  Circ. Ilhas       Vidro      3582          2.0       1.0   
926     2024-01-03  Circ. Ilhas       Vidro      6531          2.0       1.0   
1336    2024-01-03  Circuito 07  Embalagens      3870          2.0       1.0   
...            ...          ...         ...       ...          ...       ...   
362282  2024-12-31  Circuito 12       Vidro      4391          4.0       1.0   
362323  2024-12-31  Circuito 12       Vidro      4411          5.0       1.0   
362349  2024-12-31  Circuito 12       Vidro      4299          4.0       1.0   
362351  2024-12-31  Circuito 12       Vidro      4396          5.0       4.0   
362352  2024-12-31  Circuito 12       Vidro      4295          4.0       1.0   

        Recolhido  Observacao    ultima

In [34]:
maior = resultado_df[resultado_df["Nivel_inicio"] < resultado_df["Nivel_fim"]]
print(maior)

              Data     Circuito        Tipo  local_id Nivel_inicio Nivel_fim  \
845     2024-01-03  Circ. Ilhas       Papel      3844          2.0       5.0   
886     2024-01-03  Circ. Ilhas       Vidro      3864          1.0       2.0   
888     2024-01-03  Circ. Ilhas       Vidro      3865          3.0       4.0   
896     2024-01-03  Circ. Ilhas       Vidro      3929          4.0       5.0   
899     2024-01-03  Circ. Ilhas       Vidro      3664          2.0       3.0   
...            ...          ...         ...       ...          ...       ...   
362211  2024-12-31  Circuito 12  Embalagens      4415          3.0       4.0   
362212  2024-12-31  Circuito 12  Embalagens      4417          3.0       4.0   
362213  2024-12-31  Circuito 12  Embalagens      4418          1.0       2.0   
362347  2024-12-31  Circuito 12       Vidro      4294          1.0       3.0   
362350  2024-12-31  Circuito 12       Vidro      4296          1.0       4.0   

        Recolhido  Observacao    ultima

In [36]:
igual = resultado_df[resultado_df["Nivel_inicio"] == resultado_df["Nivel_fim"]]
print(igual)

              Data     Circuito   Tipo  local_id Nivel_inicio Nivel_fim  \
846     2024-01-03  Circ. Ilhas  Papel      3865          4.0       4.0   
849     2024-01-03  Circ. Ilhas  Papel      3734          4.0       4.0   
852     2024-01-03  Circ. Ilhas  Papel      3685          3.0       3.0   
856     2024-01-03  Circ. Ilhas  Papel      6568          3.0       3.0   
858     2024-01-03  Circ. Ilhas  Papel      6565          3.0       3.0   
...            ...          ...    ...       ...          ...       ...   
362359  2024-12-31  Circuito 12  Vidro      6456          1.0       1.0   
362360  2024-12-31  Circuito 12  Vidro      4310          1.0       1.0   
362361  2024-12-31  Circuito 12  Vidro      4370          4.0       4.0   
362362  2024-12-31  Circuito 12  Vidro      4344          1.0       1.0   
362363  2024-12-31  Circuito 12  Vidro      4340          1.0       1.0   

        Recolhido  Observacao     ultima_recolha ultima_observacao deltas  
846             0      

In [47]:
# Substituir células vazias ou pd.NA por string "NaN" (para visualização) nas colunas de níveis
resultado_df["Nivel_inicio"] = resultado_df["Nivel_inicio"].apply(lambda x: "NaN" if pd.isna(x) else x)
resultado_df["Nivel_fim"] = resultado_df["Nivel_fim"].apply(lambda x: "NaN" if pd.isna(x) else x)

In [ ]:
resultado_df['ultima_recolha']=pd.to_timedelta(resultado_df['ultima_recolha'])
resultado_df['ultima_observacao']=pd.to_timedelta(resultado_df['ultima_observacao'])

In [57]:
# Substituir células vazias ou pd.NA por string "NaN" (para visualização) nas colunas de níveis
resultado_df["ultima_recolha"] = resultado_df["ultima_recolha"].apply(lambda x: "NaN" if pd.isna(x) else x)
resultado_df["ultima_observacao"] = resultado_df["ultima_observacao"].apply(lambda x: "NaN" if pd.isna(x) else x)

In [50]:
resultado_df["deltas"] = resultado_df["deltas"].apply(lambda x: "NaN" if pd.isna(x) else x)

Exemplos (Confirmar com a tabela_dados_nivel_recolha_ordenado_v2)

Caso 1 (Não foi observado nem recolhido)

No dataset no dia 2024-03-18, no Circ. Ilhas o ponto 6565 não foi observado(pois não aparece nesse dia, nesse circuito) nem recolhido.
O seu nivel inicial vai ser o nivel mais atualizado até à data (ultino nivel foi no dia 2024-03-16) e como o não houve observação nem recolha
o seu nivel final manteve-se.


Caso 2 (Foi observado e recolhido)

No dataset no dia 2024-03-21, no Circuito 01 o ponto 3232 foi recolhido, logo também foi observado. 
O seu nível inicial vai ser o  o nivel mais atualizado até à data (ultino nivel foi no dia 2024-03-19, nivel 2) e  o nivel final é o nivel 
no momento da recolha (nível 5).

No dataset no dia 2024-03-21, no Circuito 01 o ponto 4228 foi recolhido, logo também foi observado. 
O seu nível inicial vai ser o  o nivel mais atualizado até à data (ultino nivel foi no dia 2024-03-19, nivel 2) e  o nivel final é o nivel 
no momento da recolha (NaN).

Caso 3 (Foi observado e não foi recolhido)

No dataset no dia 2024-03-19, no Circuito 03, tipo Embalagens, o ponto 4230 foi observado, mas nao foi recolhido 
O seu nível inicial vai ser o  o nivel mais atualizado até à data (ultino nivel foi no dia 2024-03-18, nivel 2) e o nivel final é o nivel 
no momento da observação (nível 1).




In [59]:
# Guardar ou visualizar
resultado_df.to_csv("TabelaObservacaoRecolhido.csv", index=False)


## Descrição das colunas

Nivel_inicio = Para o ponto, é o nível mais atualizado que se sabe antes desse dia começar (começarem a fazer qualquer rota). Caso o nível mais atualizado seja NaN, é colocado lá o ultimo valor diferente de NaN. Caso só haja NaN, coloca-se NaN

Nivel_fim = Quando o dia acaba (quando as rotas todas para esse dia foram feitas), é o nível que ficou registado. Se para esse ponto houve recolha, o nível aqui presente será o nivel a que estava aquando da recolha. Se houve observação, mas não houve recolha, o nível que vai se colocar é o nível aquando da observação. Se neste caso o nível for NaN, coloca-se o ultimo não NaN. Se não houve observação nem recolha, o nivel de fim vai ser igual ao nível de inicio.

Recolhido = Se o ponto foi recolhido nesse Dia, Cirucito, e Tipo. (0 não foi, 1 foi) 

Observacao = Se o houve algum registo do ponto para esse dia no ficheiro a que deu origem a esta tabela (1 se houve, 0 se não)

ultima_observacao = Representa o tempo que passou desde a última observação até ao início do dia atual (aqui é procurado em todos os circuitos)

ultima_recolha = Representa o tempo que passou desde a última recolha até ao início do dia atual (só é procurado no circuito em questão)

deltas = diferença entre nivel_fim - nivel_inicio